In [1]:
!python -V

Python 3.12.1


In [2]:
#mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


#mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")


2025/12/16 02:34:15 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/16 02:34:15 INFO mlflow.store.db.utils: Updating database tables
2025/12/16 02:34:15 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/16 02:34:15 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/16 02:34:15 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/16 02:34:15 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred,) #squared=False

60.19766156850382

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Bhuvan")

    mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
    mlflow.log_param("valid-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)    #, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

🏃 View run luminous-sponge-311 at: http://127.0.0.1:5000/#/experiments/1/runs/d571e671547f4fbcbfbc938987635c48
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)    #, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:34:23] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.76589                          
[1]	validation-rmse:11.35095                          
[2]	validation-rmse:10.96639                          
[3]	validation-rmse:10.61048                          
[4]	validation-rmse:10.28168                          
[5]	validation-rmse:9.97825                           
[6]	validation-rmse:9.69722                           
[7]	validation-rmse:9.43895                           
[8]	validation-rmse:9.20017                           
[9]	validation-rmse:8.98171                           
[10]	validation-rmse:8.78094                          
[11]	validation-rmse:8.59688                          
[12]	validation-rmse:8.42867                          
[13]	validation-rmse:8.27402                          
[14]	validation-rmse:8.13301                          
[15]	validation-rmse:8.00363                          
[16]	validation-rmse:7.88580                          
[17]	validation-rmse:7.77854                          
[18]	valid

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:35:49] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.07939                                                  
[1]	validation-rmse:6.95146                                                  
[2]	validation-rmse:6.65717                                                  
[3]	validation-rmse:6.55840                                                  
[4]	validation-rmse:6.51403                                                  
[5]	validation-rmse:6.49556                                                  
[6]	validation-rmse:6.47331                                                  
[7]	validation-rmse:6.46391                                                  
[8]	validation-rmse:6.45709                                                  
[9]	validation-rmse:6.45357                                                  
[10]	validation-rmse:6.45047                                                 
[11]	validation-rmse:6.44541                                                 
[12]	validation-rmse:6.44248                                    

...

[0]	validation-rmse:11.24049                                                   
[1]	validation-rmse:10.42280                                                   
[2]	validation-rmse:9.74095                                                    
[3]	validation-rmse:9.17231                                                    
[4]	validation-rmse:8.70981                                                    
[5]	validation-rmse:8.32136                                                    
[6]	validation-rmse:8.00121                                                    
[7]	validation-rmse:7.74500                                                    
[8]	validation-rmse:7.53300                                                    
[9]	validation-rmse:7.35950                                                    
[10]	validation-rmse:7.22150                                                   
[11]	validation-rmse:7.10808                                                   
[12]	validation-rmse:7.01157            

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:43:40] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:10.61953                                                    
[2]	validation-rmse:9.99115                                                     
[3]	validation-rmse:9.45688                                                     
[4]	validation-rmse:9.00490                                                     
[5]	validation-rmse:8.62454                                                     
[6]	validation-rmse:8.30562                                                     
[7]	validation-rmse:8.03912                                                     
[8]	validation-rmse:7.81614                                                     
[9]	validation-rmse:7.63148                                                     
[10]	validation-rmse:7.47758                                                    
[11]	validation-rmse:7.35054                                                    
[12]	validation-rmse:7.24553                                                    
[13]	validation-rmse:7.15878

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:44:39] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.68706                                                     
[1]	validation-rmse:7.37459                                                     
[2]	validation-rmse:6.89693                                                     
[3]	validation-rmse:6.72095                                                     
[4]	validation-rmse:6.64215                                                     
[5]	validation-rmse:6.60983                                                     
[6]	validation-rmse:6.59026                                                     
[7]	validation-rmse:6.57540                                                     
[8]	validation-rmse:6.56780                                                     
[9]	validation-rmse:6.56372                                                     
[10]	validation-rmse:6.55967                                                    
[11]	validation-rmse:6.55361                                                    
[12]	validation-rmse:6.54897

In [22]:
mlflow.xgboost.autolog(disable=True)

In [23]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)    #, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:45:19] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2025/12/16 02:47:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run efficient-mule-118 at: http://127.0.0.1:5000/#/experiments/1/runs/9d15362c70444ed78c8db9fd468d6e73
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [24]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor , GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
        mlflow.log_param("valid-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)    #, squared=False)
        mlflow.log_metric("rmse", rmse)
        

🏃 View run bald-rat-926 at: http://127.0.0.1:5000/#/experiments/1/runs/1c72cfc8b206433fab44a915a652a853
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run worried-conch-333 at: http://127.0.0.1:5000/#/experiments/1/runs/6da0e2dc163448dd96e7abe2eb544b3c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run flawless-smelt-145 at: http://127.0.0.1:5000/#/experiments/1/runs/b4a4e580d9d5417ebdd90fb5e1d0d662
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


🏃 View run zealous-pug-718 at: http://127.0.0.1:5000/#/experiments/1/runs/17009383fb544e68bc93d4b4bf1cafc2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
